Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [7]:
from random import choices

import lab9_lib

In [8]:
fitness = lab9_lib.make_problem(10)
for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")
 *
print(fitness.calls)

01000000100101100011000010000110011111011110011001: 7.34%
10111001000111010101111010101111110110111100001110: 23.33%
00001010001001100001100110010011011010110111001100: 15.34%
11100101100100110100101001011100110101100000010011: 9.11%
01100111100001001000101111100000101010010000110000: 7.33%
01110000101111000010111111011011011010010000001010: 23.34%
00011111010001001000101110110100000111011001011101: 9.11%
00111101110111100100000000011010100010110000000011: 7.33%
10000100110011001010100101010100011101110010100101: 7.33%
01001010111001101100110000000011001011111000011010: 15.33%
10
